In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('drive/MyDrive/DL202010821/Attention')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 1. 데이터 읽고 전처리

In [4]:
lines = pd.read_csv('dataset/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 232736


In [5]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:30000] # 3만개만 사용
lines.sample(10)

,src,tar
26742,I don't want that.,Je ne veux pas de ça !
1167,I saw him.,Je le vis.
14284,Are drinks free?,Est-ce que les boissons sont gratuites ?
25439,Bring me my shoes.,Apportez-moi mes chaussures !
1973,How absurd!,Quelle absurdité !
24108,We make mistakes.,Nous commettons des erreurs.
23435,Tom doesn't know.,Tom ne sait pas.
25739,Do you like trips?,Aimes-tu les voyages ?
9259,That's a tree.,C’est un arbre.
4292,Tom was sad.,Tom était triste.


In [6]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n') #target 데이터에 대해서만 시작과 끝을 명시해줌으로써, 디코더가 알아차릴 수 있게 함
lines.sample(10)

,src,tar
22661,She made a scene.,\t Elle a fait une scène. \n
20499,I can't join you.,\t Je ne peux me joindre à vous. \n
5965,Keep digging.,\t Ne t'arrête pas de creuser. \n
12495,Many fish died.,\t De nombreux poissons périrent. \n
6440,This is cool.,\t C'est sympa. \n
29634,They said it's OK.,\t Elles ont dit que ça convenait. \n
16223,I'm intoxicated.,\t Je suis ivre. \n
5004,Have a donut.,\t Prenez un beignet. \n
20330,How does it work?,\t Comment cela fonctionne-t-il ? \n
21983,It was a mistake.,\t Ça a été une erreur. \n


In [7]:
# character 그 자체를 넣을 수 없으므로 embedding vector 진행
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)


source 문장의 char 집합 : 77
target 문장의 char 집합 : 102


In [8]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])


['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [9]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 3

In [10]:
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])


source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [11]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])


target 문장의 정수 인코딩 : [[1, 3, 46, 50, 3, 4, 3, 2], [1, 3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [1, 3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [1, 3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [1, 3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


In [12]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])


target 문장 레이블의 정수 인코딩 : [[3, 46, 50, 3, 4, 3, 2], [3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


In [13]:
#batch_size를 묶어서 돌려야 하므로(일일이 하나하나 돌리면 cost가 많이 드므로) 덩어리채로 넣어줌 -> padding
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)


source 문장의 최대 길이 : 18
target 문장의 최대 길이 : 61


In [14]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)


# 2. 모델 학습

In [15]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Attention
from tensorflow.keras.models import Model
import numpy as np


In [20]:
encoder_inputs = Input(shape=(None, src_vocab_size))
# encoder는 return_state를 설정한 LSTM 선언
encoder_lstm = LSTM(units=256, return_state = True) #hidden_state와 cell_state 값을 뒤로 넘겨줘야 하므로 True로

# encoder의 출력 (encoder_outputs, state_h, state_c)에 저장
enconder_ouputs, state_h, state_c = encoder_lstm(encoder_inputs) # -> encoder

# LSTM은 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]



In [22]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
# decoder는 return_state와 return sequence를 설정한 LSTM 선언
decoder_lstm = LSTM(units=256, return_sequences = True,return_state=True) #many2many/many2one -> return_sequences가 설정


# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
seq2seq_outputs = decoder_softmax_layer(decoder_outputs)

#모델 선언
model = Model([encoder_inputs, decoder_inputs], seq2seq_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")


In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, None, 77)]           0         []                            
                                                                                                  
 input_7 (InputLayer)        [(None, None, 102)]          0         []                            
                                                                                                  
 lstm_3 (LSTM)               [(None, 256),                342016    ['input_4[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                            

In [24]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)


Epoch 1/40
375/375 [==============================] - 10s 15ms/step - loss: 1.1141 - val_loss: 0.9613
Epoch 2/40
375/375 [==============================] - 4s 11ms/step - loss: 0.7258 - val_loss: 0.8045
Epoch 3/40
375/375 [==============================] - 5s 12ms/step - loss: 0.6355 - val_loss: 0.7299
Epoch 4/40
375/375 [==============================] - 4s 10ms/step - loss: 0.5856 - val_loss: 0.6866
Epoch 5/40
375/375 [==============================] - 4s 11ms/step - loss: 0.5476 - val_loss: 0.6513
Epoch 6/40
375/375 [==============================] - 5s 13ms/step - loss: 0.5160 - val_loss: 0.6166
Epoch 7/40
375/375 [==============================] - 4s 11ms/step - loss: 0.4889 - val_loss: 0.5899
Epoch 8/40
375/375 [==============================] - 4s 11ms/step - loss: 0.4671 - val_loss: 0.5709
Epoch 9/40
375/375 [==============================] - 5s 13ms/step - loss: 0.4489 - val_loss: 0.5493
Epoch 10/40
375/375 [==============================] - 4s 11ms/step - loss: 0.4331 - val_l

# 3. 모델 결과 확인

In [25]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)


In [26]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용
# decoder_lstm을 사용하면서 state_h, state_c 같이 저장하기
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)


In [27]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())


In [28]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


In [29]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력


1/1 [==============================] - 0s 21ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Pars. 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Aidez-nous. 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Décampez ! 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Va chercher ! 
1/1 [==============================] - 0s 18ms/step
-----------------------------------
입력 문장: Forget me.
정답 문장: Oublie-moi. 
번역 문장: Oubliez-le. 
